In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/'

%cd "{root_dir}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [2]:
!git clone https://github.com/xelav/Insider-Threat-CERT /content/repo

import sys
sys.path.append("/content/repo")

Cloning into '/content/repo'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 181 (delta 118), reused 133 (delta 78), pack-reused 0
Receiving objects: 100% (181/181), 1.37 MiB | 1.19 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [20]:
%cd "/root/repo"
!git pull
%cd "{root_dir}"

/root/repo
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 18 (delta 14), reused 18 (delta 14), pack-reused 0
Unpacking objects: 100% (18/18), done.
From https://github.com/xelav/Insider-Threat-CERT
   393cde3..45362c8  master     -> origin/master
Updating 393cde3..45362c8
error: Your local changes to the following files would be overwritten by merge:
	src/cnn_trainer.py
Please commit your changes or stash them before you merge.
Aborting
/content/drive/My Drive


In [3]:
!pip install pytorch-ignite
%pip install wandb -q

import wandb
wandb.login()

     |████████████████████████████████| 112kB 9.9MB/s 
     |████████████████████████████████| 1.4MB 24.7MB/s 
     |████████████████████████████████| 102kB 14.3MB/s 
     |████████████████████████████████| 112kB 61.3MB/s 
     |████████████████████████████████| 460kB 62.7MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [23]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
# from tqdm import tqdm_notebook as tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.cnn_trainer import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
output_dir = Path(f'{root_dir}/Datasets/CERT_output')
answers_dir = Path(f"{root_dir}/Datasets/CERT/answers")
main_answers_file = answers_dir / "insiders.csv"

# lstm_checkpoint = output_dir / 'checkpoints/lstm/final2-nll/final_model_3040.pth'
lstm_checkpoint = output_dir / 'checkpoints/lstm/base-veryfinal2/best_model_498_non_pad_accuracy=0.7740616312925708.pth'
assert(lstm_checkpoint.is_file())

run_name = 'cnn/relu-wadb-test2'
log_dir = output_dir / 'logs' / run_name
checkpoint_dir = output_dir / 'checkpoints' / run_name

assert(not log_dir.is_dir())
assert(not checkpoint_dir.is_dir())

# if log_dir.is_dir():
#     shutil.rmtree(log_dir)
# if checkpoint_dir.is_dir():
#     shutil.rmtree(checkpoint_dir)

In [0]:
params = get_params()
params['model']['lstm_encoder']['embedding_size'] = None

train_params = params['train']['cnn_classifier']

device = 'cuda'

In [0]:
actions, targets = CertDataset.prepare_dataset(
    output_dir / 'aggregated.pkl',
    main_answers_file,
    min_length=params['min_seq_length'],
    max_length=params['seq_length'])

cert_dataset = CertDataset(actions, targets)
train_loader, val_loader = create_data_loaders(
    cert_dataset,
    validation_split=0.3,
    random_seed=0,
    batch_size=train_params['batch_size'])

In [0]:
def calculate_weights(targets, weight_beta=0.1):

    anomalous_num = targets.sum()
    overall_num = targets.shape[0]
    non_anomalous_num = overall_num - anomalous_num

    anomalous_weight = 1 + (non_anomalous_num - anomalous_num) / (weight_beta * non_anomalous_num)

    weight = torch.tensor([1, anomalous_weight]).float()

    return weight

class_weights = calculate_weights(targets, train_params['weighting_beta']).to(device)

In [0]:
model = InsiderClassifier(params['model'], lstm_checkpoint)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters())

train_engine = create_supervised_trainer(
                                        model, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch,
                                        log_dir=(output_dir / 'logs' / 'cnn' / run_name).as_posix(),
                                        checkpoint_dir=output_dir / 'checkpoints' / 'cnn' / run_name
                                       )

val_engine = create_supervised_evaluator(
        model, device=device,
        prepare_batch=prepare_batch,
        metrics={},
        criterion=criterion,
        log_dir=log_dir.as_posix(),
)

@train_engine.on(Events.STARTED)
def initial_validation(trainer):
    print('Initial validation run:')
    val_engine.train_epoch = 0
    val_engine.run(val_loader)

    wandb.run.summary["best_val_roc_auc"] = val_engine.state.metrics['roc_auc']
    wandb.run.summary["best_val_loss"] = val_engine.state.metrics['loss']


@train_engine.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    val_engine.train_epoch = train_engine.state.epoch
    val_engine.run(val_loader)

    if val_engine.state.metrics['roc_auc'] > wandb.run.summary["best_val_roc_auc"]:
        wandb.run.summary["best_val_roc_auc"] = val_engine.state.metrics['roc_auc']
        wandb.run.summary["best_val_loss"] = val_engine.state.metrics['loss']
        wandb.run.summary["best_train_roc_auc"] = train_engine.state.metrics['roc_auc']
        wandb.run.summary["best_train_loss"] = train_engine.state.metrics['loss']


In [36]:
wandb.init(project="cert-cnn", config=params)
wandb.watch(model)

In [0]:
torch.manual_seed(train_params['manual_seed'])
train_engine.run(train_loader, max_epochs=train_params['num_epochs'])

Initial validation run:
Validation Results - Avg loss: 0.730658, ROC AUC: 0.566782
Train Results - Avg loss: 0.040293, ROC AUC: 0.517635
Validation Results - Avg loss: 0.024023, ROC AUC: 0.749399
Train Results - Avg loss: 0.026686, ROC AUC: 0.569166
Validation Results - Avg loss: 0.023824, ROC AUC: 0.798583
Train Results - Avg loss: 0.024869, ROC AUC: 0.785421
Validation Results - Avg loss: 0.021582, ROC AUC: 0.889102
Train Results - Avg loss: 0.021866, ROC AUC: 0.885737
Validation Results - Avg loss: 0.019760, ROC AUC: 0.895779
Train Results - Avg loss: 0.020231, ROC AUC: 0.914306
Validation Results - Avg loss: 0.018998, ROC AUC: 0.903483


In [0]:
%load_ext tensorboard
%tensorboard --logdir "{log_dir}"

In [0]:
%cd "{output_dir}"
!ls

/content/drive/My Drive/Datasets/CERT_output
aggregated.pkl	     device_preprocessed.csv  logon_preprocessed.csv
checkpoint_1000.pth  email_preprocessed.csv   logs
checkpoints	     file_preprocessed.csv


In [0]:
# log_zip =  output_dir / 'logs' / (run_name + '_logs.zip')
log_zip = log_dir.stem + '_logs.zip'
%cd "{output_dir / 'logs'}"
!zip -r "$log_zip" "$run_name"
files.download(log_zip)

/content/drive/My Drive/Datasets/CERT_output/logs
  adding: cnn/relu-batchnorm/ (stored 0%)
  adding: cnn/relu-batchnorm/validation/ (stored 0%)
  adding: cnn/relu-batchnorm/validation/events.out.tfevents.1586931120.450bbb643380.121.3 (deflated 19%)
  adding: cnn/relu-batchnorm/validation/events.out.tfevents.1586931242.450bbb643380.121.5 (deflated 70%)


# Prediction exploration

In [0]:
model.eval()

InsiderClassifier(
  (lstm_encoder): LSTM_Encoder(
    (lstm_encoder): LSTM(64, 40, num_layers=3, batch_first=True, dropout=0.5)
    (dropout): Dropout(p=0.5)
    (decoder): Linear(in_features=40, out_features=64, bias=True)
    (log_softmax): LogSoftmax()
  )
  (sigmoid): Sigmoid()
  (cnn_classifier): CNN_Classifier(
    (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (linear): Linear(in_features=32000, out_features=2, bias=True)
    (softmax): LogSoftmax()
  )
)

In [0]:
it = iter(val_loader)
for batch in it:
    x, y = prepare_batch(batch)
    if y.sum() > 0:
        break

In [0]:
ind = (y==1).nonzero()[0]
ind

tensor([46])

In [0]:
with torch.no_grad():
    res = model(x.to(device))[ind]
F.softmax(res)

C:\Anaconda3\envs\python3.6-torch\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


tensor([[0.8219, 0.1781]], device='cuda:0')